In [1]:
import sys
sys.executable

'c:\\users\\alind\\appdata\\local\\programs\\python\\python39\\python.exe'

In [3]:
"""
So now we have turtle files which describe readings.
But that information isn't so complete or accurate.
So we need to ask some bibliographic databases online
for better information, so that we can enhance our graph.
"""

from rdflib.namespace import DC, DCTERMS, FOAF, RDF, OWL
from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib import Namespace
import rdflib
import click
import sys
import requests
import logging
import json


# @click.command()
# @click.argument('turtlefile', nargs=1)



In [4]:
turtleFile = "data/coursesAndTexts.ttl"

turtleFile= "data/coursesAndTexts_edit_2.ttl"

def queryCrossRef(title, author=None):
    """
    Let's look up data from CrossRef. Here's the form we'll create,
    from their documentation here: https://github.com/CrossRef/rest-api-doc#resource-components
    https://api.crossref.org/works?query.author=richard+feynman
    """
    logging.info(f"Querying {title} by {author}")
    url = "https://api.crossref.org/works"
    params = {"query.bibliographic": title}
    if author:
        params['query.author'] = author
    resp = requests.get(url, params=params)
    if resp.ok:
        decoded = json.loads(resp.text)
        if 'message' in decoded:
            if 'items' in decoded['message']:
                print(decoded['message']['items'][0])
            else:
                logging.error("Can't find items.")
        else:
            logging.error("Can't find the message.")
    else:
        logging.error("Response not ok. Response: {resp}")




In [5]:
# https://pypi.org/project/edit-distance/
import edit_distance

In [7]:
import sys
sys.version

'3.9.1 (tags/v3.9.1:1e5d33e, Dec  7 2020, 17:08:21) [MSC v.1927 64 bit (AMD64)]'

In [6]:
import nltk
# edit distance
# https://github.com/JonathanReeve/data-ethics-literature-review/blob/main/turtleize/enhanceBibliography.py

def editRatio(textA, textB):
    """
    Computes the ratio of edit distances between two titles,
    in order to measure title similarity.
    """
    return nltk.edit_distance(textA, textB) / (len(textA + textB)/2)



In [7]:
# use title and author to query Arxiv metadata

import urllib
import feedparser
import urllib.request

base_url = 'http://export.arxiv.org/api/query?';

# my arxiv query fn
def queryArxiv(title, author=None):
    
    stringsplit = title.split()
    st= ""
    for s in stringsplit:
        st =st+s+"+"
    
    st = st[:-1]
    
    # add title to query
    query = 'search_query='+st+'&searchtype=title'
    
    # if there is author, add the author to search with +AND+  plus searchtype=author
    if author != None:
        # add to query
        query = query+'+AND+'
        # add author
        authorsplit = author.split()
        st = ""
        for s in authorsplit:
            st = st+s+"+"
        st = st[:-1]
        query = query+st +'&searchtype=author'
        
    # query=The+Great+A.I&searchtype=title
    urlquery = base_url+query
    urlquery.encode('utf-8') # 7/
    print("url query", urlquery)
    with urllib.request.urlopen(urlquery) as url:
        response = url.read()

    # parse the response using feedparser
    feed = feedparser.parse(response)
    
    editdists = []
    
    aids =[]
    apublished =[]
    atitles=[]
    aauthors=[]
    aabstracts=[]
    
    # add to the graph the fields
    for entry in feed.entries:
        """print('arxiv-id: %s' % entry.id.split('/abs/')[-1])
        print('Published: %s' % entry.published)
        print('Title:  %s' % entry.title)"""
        arxiv_id = entry.id
        date = entry.published
        atitle = entry.title
        author_string = entry.author
        abstract = entry.summary
        
        # calc edit dist
        edist = editRatio(title, atitle)
        
        # add to arrays
        editdists.append(edist)
        aids.append(arxiv_id)
        apublished.append(date)
        atitles.append(atitle)
        aauthors.append(author_string)
        aabstracts.append(abstract)
    
        #g.add((item, dcterms:title, title))

    # determine which article is best
    minindex = np.argmin(editdists)
    #return
    """title.encode("utf8")
    author_string.encode("utf8")
    arxiv_id.encode("utf8")
    date.encode("utf8")
    abstract.encode("utf8")"""
    #date.replace("u'", "'")
    rtitle = atitles[minindex]
    rauthor_string = aauthors[minindex]
    rarxiv_id = aids[minindex]
    rabstract = aabstracts[minindex]
    
    return(rtitle, rauthor_string, rarxiv_id, rabstract)

 #add data to graph function


# choose correct article with edit distance

In [51]:
(t1, t2, t3, t4)= queryArxiv('atomic')

arxiv-id: 1207.3006v1
Published: 2012-07-12T16:04:13Z
Title:  Excitation Suppression Due to Interactions Between Microwave-Dressed
  Rydberg Atoms
arxiv-id: 1207.6921v1
Published: 2012-07-30T12:49:45Z
Title:  Dimer-atom-atom recombination in the universal four-boson system
arxiv-id: 1410.7957v1
Published: 2014-10-29T12:48:48Z
Title:  Effective oscillator strength distributions of spherically symmetric
  atoms for calculating polarizabilities and long-range atom-atom interactions
arxiv-id: 1912.12035v1
Published: 2019-12-27T09:20:05Z
Title:  Evolution of energy and angular distributions of sputtered atoms with
  variation of atomic number of single crystal target
arxiv-id: 2101.03565v1
Published: 2021-01-10T15:18:38Z
Title:  Improving Efficiency of Sympathetic Cooling in Atom-Ion and Atom-Atom
  Confined Collisions
arxiv-id: physics/0505131v1
Published: 2005-05-19T12:50:08Z
Title:  Long range intermolecular forces in triatomic systems: connecting the
  atom-diatom and atom-atom-atom rep

In [8]:
import numpy as np

In [1]:
# Sample queries 

# https://bibliontology.com/content/article.html

# https://rdflib.readthedocs.io/en/stable/rdf_terms.html

"""select ?authorLast ?authorGiven
where"""

# question mark where do not know 
"""?myText dc:creator ?authorID .
?authorID a foaf:person .
?authorID givenName .
?authorID lastName ?authorLast ."""

authorquery1 = """SELECT ?name
WHERE {
    ?person foaf:name ?name .
}
"""

authorquery = """select ?givenName ?surname where{
 var(id) a foaf:Person .
}"""

# draft pull request 

SyntaxError: invalid syntax (<ipython-input-1-88d971a990ce>, line 7)

In [9]:
# necessary function to remove non-string symbols
def ExtractAlphanumeric(InputString):
    from string import ascii_letters, digits
    return "".join([ch for ch in InputString if ch in (ascii_letters + digits)])

In [14]:
# My main
from rdflib import URIRef
from rdflib import Literal
def main():
    g = rdflib.Graph()
    g.load(turtleFile, format="ttl")
    #    for item in g:
    #     print(item)
    g.bind('z', 'http://www.zotero.org/namespaces/export#')
    g.bind('dcterms', 'http://purl.org/dc/terms/')
    g.bind('foaf', 'http://xmlns.com/foaf/0.1/')
    # This only works if we have the title and author.
    # data = g.query("""select distinct ?id ?title ?authorFirst ?authorLast where {
    #     ?id a z:UserItem .
    #     ?id res:resource ?doc .
    #     ?doc dcterms:title ?title .
    #     ?doc dcterms:creator ?author .
    #     ?author foaf:givenName ?authorFirst .
    #     ?author foaf:surname ?authorLast .
    # }""")
    # itemID, title, authorFirst, authorLast = list(data)[0]
    # print(title, authorFirst, authorLast)
    # This will work even if we don't have an author
    data = g.query("""select distinct ?id ?title ?authorFirst ?authorLast where {
        ?id a z:UserItem .
        ?id res:resource ?doc .
        ?doc dcterms:title ?title .
        OPTIONAL {
          ?doc dcterms:creator ?author .
          ?author foaf:givenName ?authorFirst .
          ?author foaf:surname ?authorLast .
        }
    }""")
    resultsDict = {}
    for result in data:
        itemID, title, authorFirst, authorLast = result
        print(itemID, title, authorFirst, authorLast)
        if authorFirst is not None and authorLast is not None:
            author = f"{authorFirst} {authorLast}"
        else:
            author = None
        itemID = str(itemID)
        if itemID in resultsDict:
            continue # Only take the first one for each ID
        else:
            resultsDict[itemID] = (title, author)
    for itemID, titleAuthor in resultsDict.items():
        title, author = titleAuthor
        print("orig title", title)
        # replace with my function
        ss = title.split()
        newstring=""
        for st in ss:
            st1 = ExtractAlphanumeric(st)
            newstring=newstring+st1+" "
        newstring=newstring[:-1]
        
        
        (t1, t2, t3, t4) = queryArxiv(newstring)# title, author, arxiv id, abstract
        
        if author != None:
            # query Author 
            aq = "select ?givenName ?surname where{ "
            aq = aq + author
            aq =aq + " a foaf:Person .} "
            
            qauthor = g.query(aq)
        
            print("qauthor ", quathor)
            (t1, t2, t3, t4) = queryArxiv(newstring, qauthor)
        
        
        
        print("item id is", itemID)
        # url to rdflib term : wrap in rdf (rdflib documentation)
        uitemID = URIRef(itemID)
        
        print("best title:", t1)
        print("best author:", t2) # dc terms creator
        #g.add((uitemID, "dcterms:title", t1))
        #g.add((uitemID, "dcterms:abstract", t4))
        g.add((uitemID, URIRef('http://purl.org/dc/terms/title'), Literal(t1))) #URIRef(t1)))
        g.add((uitemID, URIRef('http://purl.org/dc/terms/creator'), Literal(t2)))
        g.add((uitemID, URIRef('http://purl.org/dc/terms/abstract'), Literal(t4))) # URIRef(t4)))

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    main()

https://data-ethics.tech/text/4MFTEYZK Transparent, explainable, and accountable AI for None None
https://data-ethics.tech/text/4K4B5FLS When the Algorithm Itself is a Racist: Diagnosing Ethical Harm in the Basic Components of Software None None
https://data-ethics.tech/text/4KCWRSUW The Great A.I None None
https://data-ethics.tech/text/4M5D2C6G The Tangled Story Behind Trump’s False Claims Of Voter Fraud None None
orig title Transparent, explainable, and accountable AI for
author None
True


TypeError: can only concatenate str (not "NoneType") to str

In [76]:
 #test alphanumeric
ss = string.split()
newstring=""
for st in ss:
    st1 = ExtractAlphanumeric(st)
    newstring=newstring+st1+" "

In [ ]:
# write graph to output file
g.serialize(destination='output.txt', format='turtle')
